In [ ]:
import cv2

# Open a window to show the camera feed
cv2.namedWindow("ViT Model Test")

# Initialize the camera (0 usually refers to the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        break

    # Display the resulting frame
    cv2.imshow("ViT Model Test", frame)

    # Press 'q' to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows() 

: 

In [2]:
from transformers import ViTForImageClassification
model = ViTForImageClassification.from_pretrained('ChrisGuarino/cat_ds')

In [ ]:
import cv2
import torch
from ModelTrain import model, preprocess_image  # Replace with your model and preprocessing

# Load your trained model
model = YourModel()
model.eval()  # Set the model to evaluation mode

# Initialize the camera
cap = cv2.VideoCapture(0)  # 0 is usually the default camera

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the image
        input_tensor = preprocess_image(frame)

        # Make a prediction
        with torch.no_grad():
            prediction = model(input_tensor)

        # Process the prediction (e.g., convert to readable format)
        readable_prediction = process_prediction(prediction)  # Define this based on your model output

        # Display the resulting frame with prediction
        cv2.putText(frame, readable_prediction, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Cat Recognition', frame)

        # Break the loop with 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # When everything is done, release the capture
    cap.release()
    cv2.destroyAllWindows()
